In [1]:
import torch
from ultralytics import YOLO
import supervision as sv
import cv2

print("--- Ortam Testi Başlıyor ---")

# 1. PyTorch ve GPU Kontrolü
print("\n1. PyTorch ve GPU Durumu:")
if torch.cuda.is_available():
    print(f"  CUDA mevcut: EVET (GPU algılandı)")
    print(f"  GPU Adı: {torch.cuda.get_device_name(0)}")
    print(f"  PyTorch CUDA versiyonu: {torch.version.cuda}")
else:
    print("  CUDA mevcut: HAYIR (GPU algılanmadı veya doğru kurulmadı)")
    print("  GPU desteği olmadan devam edilecek.")

# 2. Ultralytics (YOLO) Kontrolü
print("\n2. Ultralytics (YOLO) Kontrolü:")
try:
    model = YOLO("yolov8n.pt")
    print("  YOLO modeli başarıyla yüklendi (yolov8n.pt).")
    # Küçük bir test çalıştıralım (gerçek bir tespit yapmaz, sadece modelin yüklendiğini kontrol eder)
    # Modelin iç mekanizmalarını tetikler.
    test_input = torch.randn(1, 3, 640, 640) # Sahte bir giriş tensörü
    _ = model(test_input)
    print("  YOLO modeli test çağrısı başarılı.")
except Exception as e:
    print(f"  HATA: YOLO modeli yüklenirken/test edilirken bir sorun oluştu: {e}")

# 3. Supervision Kütüphanesi Kontrolü
print("\n3. Supervision Kütüphanesi Kontrolü:")
try:
    # Temel sınıfları içe aktarma
    _ = sv.Detections.empty()
    _ = sv.BoxAnnotator()
    _ = sv.LabelAnnotator()
    
    # ByteTrack'i kontrol etme
    _ = sv.ByteTrack()
    print("  Supervision Detections, Annotators ve ByteTrack başarıyla içe aktarıldı.")
except Exception as e:
    print(f"  HATA: Supervision kütüphanesi içe aktarılırken bir sorun oluştu: {e}")

# 4. OpenCV Kontrolü
print("\n4. OpenCV Kontrolü:")
try:
    _ = cv2.VideoCapture(0) # Kamerayı açmaya çalışır, hata vermezse kurulu demektir.
    print("  OpenCV başarıyla içe aktarıldı ve temel işlevler test edildi.")
except Exception as e:
    print(f"  HATA: OpenCV içe aktarılırken bir sorun oluştu: {e}")

print("\n--- Test Sonuçlandı ---")

--- Ortam Testi Başlıyor ---

1. PyTorch ve GPU Durumu:
  CUDA mevcut: EVET (GPU algılandı)
  GPU Adı: NVIDIA GeForce RTX 4060 Laptop GPU
  PyTorch CUDA versiyonu: 11.8

2. Ultralytics (YOLO) Kontrolü:


100%|██████████| 6.25M/6.25M [00:06<00:00, 1.07MB/s]


  YOLO modeli başarıyla yüklendi (yolov8n.pt).

WARNING torch.Tensor inputs should be normalized 0.0-1.0 but max value is 5.148391246795654. Dividing input by 255.
0: 640x640 (no detections), 9.8ms
Speed: 1.9ms preprocess, 9.8ms inference, 78.0ms postprocess per image at shape (1, 3, 640, 640)
  YOLO modeli test çağrısı başarılı.

3. Supervision Kütüphanesi Kontrolü:
  Supervision Detections, Annotators ve ByteTrack başarıyla içe aktarıldı.

4. OpenCV Kontrolü:
  OpenCV başarıyla içe aktarıldı ve temel işlevler test edildi.

--- Test Sonuçlandı ---


In [ ]:
#gerekli kütüphaneler 
import cv2
import os
from ultralytics import YOLO
# supervision kütüphanesini 'sv' olarak içe aktarmak yaygın
import supervision as sv 

#
# 3. Model ve Tracker Başlat
model = YOLO("yolov8n.pt")  # YOLO modeli
# ByteTrack nesnesini 'sv' kısaltması üzerinden başlatın
tracker = sv.ByteTrack()    # ByteTrack nesne izleyici

#4. Video ve Yazıcı Ayarları
video_path = "videos/v2.mp4"
output_video_path = "output/street_detection.mp4"
cap = cv2.VideoCapture(video_path)

width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)

os.makedirs("output", exist_ok=True)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# 5. Annotator ve Sayaçlar
# Annotator'ları da 'sv' üzerinden başlatın
box_annotator = sv.BoxAnnotator(thickness=2, color=sv.Color(255, 0, 0))
label_annotator = sv.LabelAnnotator(text_thickness=2, text_scale=1, text_color=sv.Color(255, 255, 255), color=sv.Color(0, 0, 0))

unique_ids = set()          # Her benzersiz kişi için ID tutar
frame_idx = 0               # Kare sayacı

#6. Kare İşleme ve Takip
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Sadece 'person' sınıfını (class_id=0) tespit etmek için filters=0 kullan
    results = model(frame, classes=0, conf=0.25)[0]
    # Detections nesnesini 'sv' üzerinden oluştur
    detections = sv.Detections.from_ultralytics(results)

    # Takip yap
    detections = tracker.update_with_detections(detections)

    # Kişi sayımı için ID'leri topla
    # detections.tracker_id, takip edilen nesnelerin ID'lerini içerir.
    # Eğer bir karede hiç nesne yoksa veya tracker_id boşsa, bu döngü çalışmaz.
    if detections.tracker_id is not None:
        for track_id in detections.tracker_id:
            if track_id not in unique_ids:
                unique_ids.add(track_id)

    # O anki karedeki kişi sayısı (eğer detections.tracker_id boşsa 0 olacaktır)
    current_count = len(detections.tracker_id) if detections.tracker_id is not None else 0
    total_count = len(unique_ids)              # Videoda tespit edilen toplam kişi sayısı

    # Kalabalık yoğunluğu: kişi sayısı / toplam alan (piksel cinsinden)
    area = width * height
    # Alan 0 ise bölme hatası vermemek için kontrol ekle
    density = (current_count / area * 1e6) if area > 0 else 0 # Normalleştirme için 1M ile çarpıldı

    # Etiketler
    # Eğer detections.tracker_id boşsa etiketleri boş liste olarak ayarlayın
    labels = [f"ID: {track_id}" for track_id in detections.tracker_id] if detections.tracker_id is not None else []
    
    frame = box_annotator.annotate(frame, detections)
    frame = label_annotator.annotate(frame, detections, labels)

    # Sayaçları ekrana yaz
    cv2.putText(frame, f"Current Count: {current_count}", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    cv2.putText(frame, f"Total Count: {total_count}", (20, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    cv2.putText(frame, f"Density: {density:.4f}", (20, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

    out.write(frame)
    frame_idx += 1

#7. Temizlik
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Toplam kişi sayısı: {total_count}")


0: 384x640 37 persons, 18.7ms
Speed: 6.9ms preprocess, 18.7ms inference, 3.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 38 persons, 17.7ms
Speed: 3.2ms preprocess, 17.7ms inference, 3.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 35 persons, 16.7ms
Speed: 3.6ms preprocess, 16.7ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 37 persons, 16.9ms
Speed: 3.5ms preprocess, 16.9ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 37 persons, 16.8ms
Speed: 3.1ms preprocess, 16.8ms inference, 5.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 38 persons, 16.9ms
Speed: 3.9ms preprocess, 16.9ms inference, 5.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 36 persons, 16.8ms
Speed: 3.1ms preprocess, 16.8ms inference, 3.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 36 persons, 22.4ms
Speed: 3.3ms preprocess, 22.4ms inference, 5.0ms postprocess per image at